In [21]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.csv_loader import CSVLoader
path = "./pull_requests_summary.csv"

import csv
csv.field_size_limit(10**6)

loader = CSVLoader(file_path=path, encoding="utf-8", csv_args={'delimiter': ','})
data = loader.load()

In [56]:
#print(data)

count = 0
for line in data:
    text = line.page_content
    if "Commit ID" in text :
        count = count+1
        print(count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [43]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=0)
chunks = text_splitter.split_documents(data)

In [15]:
for i in range(10) :
    print(f"\nChunk value in iteration {i} is : ", chunks[i])
    
last_chunk = chunks[-10:]
for j in range(len(last_chunk)) :
    print(f"\nChunk value in iteration {j} is : ", chunks[j])


Chunk value in iteration 0 is :  page_content='Commit ID: 1463a176bf652172560c3893c84b61c86e1b47f5' metadata={'source': './pull_requests_summary.csv', 'row': 0}

Chunk value in iteration 1 is :  page_content='Changes: diff --git' metadata={'source': './pull_requests_summary.csv', 'row': 0}

Chunk value in iteration 2 is :  page_content='--git a/src/main/java/com/manh/cp/appointment/appointment/constants/FeatureFlagConstants.java' metadata={'source': './pull_requests_summary.csv', 'row': 0}

Chunk value in iteration 3 is :  page_content='b/src/main/java/com/manh/cp/appointment/appointment/constants/FeatureFlagConstants.java' metadata={'source': './pull_requests_summary.csv', 'row': 0}

Chunk value in iteration 4 is :  page_content='index dd0b50b37..84bb7cd76 100644' metadata={'source': './pull_requests_summary.csv', 'row': 0}

Chunk value in iteration 5 is :  page_content='--- a/src/main/java/com/manh/cp/appointment/appointment/constants/FeatureFlagConstants.java' metadata={'source': '

In [20]:
count = 0
for chunk in chunks:
    text = chunk.page_content
    if "Commit" in text :
        count = count+1
        print(count)
        print(chunk.page_content)

1
Commit ID: 1463a176bf652172560c3893c84b61c86e1b47f5
Changes: diff --git a/src/main/java/com/manh/cp/appointment/appointment/constants/FeatureFlagConstants.java b/src/main/java/com/manh/cp/appointment/appointment/constants/FeatureFlagConstants.java
index dd0b50b37..84bb7cd76 100644
--- a/src/main/java/com/manh/cp/appointment/appointment/constants/FeatureFlagConstants.java
+++ b/src/main/java/com/manh/cp/appointment/appointment/constants/FeatureFlagConstants.java
2
Commit ID: 39c8f648fdb08e8717237ae4ffca5b181cd3fd33
Changes: diff --git a/gradle.properties b/gradle.properties
index 63e41475e..541cd2205 100644
--- a/gradle.properties
+++ b/gradle.properties
@@ -1 +1 @@
-dbChecksum=5cf61ada06975e25d78b471e00b5184e
+dbChecksum=153d228ee28c89f7182274a47fc7cd8e
3
Commit ID: 949be2ac0e957cfbe981da242384424b4786067a
Changes: diff --git a/src/component-test/java/com/manh/cp/appointment/appointment/DaylightSavings/AppointmentDaylightChangeParameterizedTest.java b/src/component-test/java/com/manh

In [44]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model= "text-embedding-3-small", dimensions=1536) #text-embedding-3-small

In [45]:

from langchain_community.vectorstores import Chroma
persist_directory_csv ='./chroma_db_csv'
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=embeddings, 
    persist_directory=persist_directory_csv, 
    collection_name="myRAG-CSV"
    )

In [64]:
from langchain.chains import LLMChain
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. 
    You are given a document with 3 columns
    first column name is Commit ID
    Second column name is Changes
    Third column name is Comments
    Your task is to generate three 
    different versions of the given user question to retrieve relevant documents from a vector 
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search. 
    Provide these alternative questions separated by newlines.
    You should scan the full document to get the answer
    Original question: {question}""",
)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

from langchain.retrievers.multi_query import MultiQueryRetriever
retriever = MultiQueryRetriever.from_llm(vector_db.as_retriever(), llm, prompt=QUERY_PROMPT)

template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Only provide the answer from the {context}, nothing else.
Add snippets of the context you used to answer the question.
"""

prompt = ChatPromptTemplate.from_template(template)

# Chain
llm_chain = ({"context": retriever, "question": RunnablePassthrough()} | prompt | llm | StrOutputParser())

# Other inputs
question = "Find the total Commit ID in the document?"

response = llm_chain.invoke(input = question)

print(response)

I don't know. 

The provided context does not contain any information regarding a "Commit ID."


In [61]:
#retriever = vector_db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.0})
retriever = vector_db.as_retriever()
question = "How many commit IDs are present in the document?"
docs = retriever.invoke(question)
for doc in docs:
    print("\n")
    print(doc)



page_content='+                            .listAppointments(context, new Query(null, 0, Integer.MAX_VALUE), null, null, null, null,
+                                    null)
+                            .getBody().getData();
+                    assertPojoEquals(list, documents);
+                    break;
+                }
+                case TIME_INTERVAL_UTILIZATION:
+                {
+                    List<Document> documents = timeIntervalUtilizationController
+                            .listTimeIntervalUtilizations(context, new Query(null, 0, Integer.MAX_VALUE), null, null,
+                                    null, null, null)
+                            .getBody().getData();
+                    assertPojoEquals(list, documents);
+                    break;
+                }
+                case DAY_LEVEL_UTILIZATION:
+                {
+                    List<Document> documents = dayLevelUtilizationController
+                            .listDayLevelUtiliz

In [69]:
from dotenv import load_dotenv, find_dotenv
founddotenv = load_dotenv(find_dotenv(), override=True) 
print("Found .env: %s", founddotenv)

Found .env: %s True


In [77]:
import os
import pandas as pd
from pandasai import Agent
from pandasai import SmartDataframe

path = "./pull_requests_summary.csv"

import csv
csv.field_size_limit(10**6)

df = pd.read_csv(path)

In [78]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
sdf = SmartDataframe(df, config={"llm": llm, "verbose": True})

In [80]:
question = 'How many Commit ID s are present'
sdf.chat(question)

{'type': 'number', 'value': 20}


20

In [93]:
question = 'Find the 2 most similar Commit Ids and state WHY they are similar'
sdf.chat(question)

{'type': 'string', 'value': 'The two most similar commit IDs are 949be2ac0e957cfbe981da242384424b4786067a and 8f0a90296125be546416d56ccc90faa7eeaa4e20 with a similarity score of 0.17. They are similar because they both involve changes to the appointment service and error definitions.'}


'The two most similar commit IDs are 949be2ac0e957cfbe981da242384424b4786067a and 8f0a90296125be546416d56ccc90faa7eeaa4e20 with a similarity score of 0.17. They are similar because they both involve changes to the appointment service and error definitions.'

In [81]:
# Use your API key to instantiate an LLM 
from pandasai import Agent 
from pandasai.llm.openai import OpenAI 
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
#llm = OpenAI() 

sdf = SmartDataframe(df, config={"llm": llm, "verbose": True})

agent = Agent(sdf)
response = agent.chat(question)
print(response)


{'type': 'number', 'value': 20}


20

In [83]:
question = 'Which Commit Id has the maximum number of Changes and how many?'
response = agent.chat(question)
print(response)

The Commit ID d44fe09aad42bdaee71ec01ad46ea9373838cc79 has the maximum number of Changes with 4954 changes.


In [91]:
question = 'which are the 2 most similar Commit IDs?'
response = agent.chat(question)
print(response)

                                  Commit_ID_1  \
145  c8ece09049aab49a3563e0270356ff252cab07ec   
151  c8ece09049aab49a3563e0270356ff252cab07ec   

                                  Commit_ID_2  Similarity_Score  
145  d44fe09aad42bdaee71ec01ad46ea9373838cc79             10302  
151  eaaee88fd31f620451b7e0d94302793da4c0f041             10149  


In [87]:
explanation = agent.explain()
print(explanation)

I analyzed the data you provided and created a program to compare different Commit IDs based on the changes they contain. By looking at the similarities in the changes made in each commit, I was able to identify the two most similar Commit IDs. The code calculates a similarity score between pairs of Commit IDs and then ranks them to find the most similar ones. This helps in understanding which commits are closely related based on the changes they introduce.
